# Fase 3.1: Construcción del Backend de IA (Python API)

En esta subfase, desarrollamos el núcleo lógico que impulsará al Chatbot. Creamos una clase (`RetailIntelligenceBackend`) capaz de orquestar los servicios de OCI para responder a dos tipos de necesidades del usuario: recomendaciones pasivas (basadas en gustos) y búsquedas activas (basadas en intención).

## Configuración del Entorno y Librerías
Preparamos el entorno de ejecución instalando los SDKs de Oracle Cloud y las librerías de Machine Learning necesarias.

In [ ]:
# Instalamos las dependencias necesarias
!pip install oci
!pip install oracledb
# !pip install scikit-surprise joblib
# !pip install "numpy<2" --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.9 MB/s eta 0:00:00


In [ ]:
#Realizamos las importaciones correspondientes
import oci
import pandas as pd
import oracledb
# from surprise import SVD, Dataset, Reader, accuracy
import os
import joblib
import array
import json
import numpy as np

### Inicialización de Clientes OCI


Configuramos la autenticación con el archivo config y levantamos los clientes para Object Storage (almacenamiento de modelos) y OCI Generative AI (generación de embeddings). Es crucial usar el `GenerativeAiInferenceClient` para poder realizar inferencias en tiempo real.

In [ ]:
# Configuración de OCI
NAMESPACE = 'ax5grcoay8ni'
BUCKET_NAME = 'Team3'
GENAI_ENDPOINT = 'https://inference.generativeai.us-chicago-1.oci.oraclecloud.com'
GENAI_COMPARTMENT_ID = 'ocid1.compartment.oc1..aaaaaaaatjpti23mkvyvggtpv6kqujtabcigjaswynnfmhvjdjtkeaqfwakq' # Compartimento creado por el equipo

# Cargamos la configuracion y los clientes (object storage y vision)
try:
    config = oci.config.from_file("config", "DEFAULT")
    # Cliente de Object Storage (para obtener el CSV)
    object_storage_client = oci.object_storage.ObjectStorageClient(config)

    # Cliente de inferencia OCI Gen AI
    genai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(
        config=config,
        service_endpoint=GENAI_ENDPOINT
    )
    print("Clientes inicializados correctamente.")
except Exception as e:
    print(f"Error al inicializar clientes: {e}")

Clientes inicializados correctamente.


## Carga de Recursos (Modelos y Datos)
El backend necesita tener acceso inmediato a los "cerebros" del sistema

### Descarga del Modelo Basado en Embeddings

El sistema emplea un modelo de recomendación embedding-based (`recommendation_model_v1.pkl`), el cual encapsula representaciones vectoriales para todo el catálogo de artículos.

Dicho modelo se descarga desde OCI Object Storage al sistema de archivos local para ser cargado en la memoria RAM. Esto minimiza la latencia en cada predicción.

In [ ]:
# Configuración del archivo
MODEL_FILENAME = 'recommendation_model_v1.pkl'
LOCAL_MODEL_PATH = f'/content/{MODEL_FILENAME}'
OBJECT_NAME_IN_BUCKET = f"models/{MODEL_FILENAME}"

try:
    # Descargamos desde Object Storage
    get_obj = object_storage_client.get_object(NAMESPACE, BUCKET_NAME, OBJECT_NAME_IN_BUCKET)

    with open(LOCAL_MODEL_PATH, 'wb') as f:
        for chunk in get_obj.data.raw.stream(1024 * 1024, decode_content=False):
            f.write(chunk)

    print(f"Modelo descargado exitosamente en: {LOCAL_MODEL_PATH}")
    print(f"Tamaño: {os.path.getsize(LOCAL_MODEL_PATH) / (1024*1024):.2f} MB")

except Exception as e:
    print(f"Error descargando modelo: {e}")

Modelo descargado exitosamente en: /content/recommendation_model_v1.pkl
Tamaño: 413.08 MB


In [ ]:
# --- CONFIGURACIÓN DE PARÁMETROS DB ---
db_config = {
    "user": "ADMIN",
    "password": "TecMonterreyTeam3",
    "dsn": "team3vectordatabase_high",
    "config_dir": "/content/wallet",
    "wallet_location": "/content/wallet",
    "wallet_password": "TecMonterreyTeam3"
}

### Conexión a Base de Datos Vectorial


Definimos la función query y la configuración de conexión hacia nuestra Autonomous Database 26ai. Esta conexión es vital para acceder al catálogo de productos (ARTICLES_MODIFIED) y realizar búsquedas de similitud en la tabla vectorial (ARTICLE_EMBEDDINGS).

In [ ]:
def query(sql, params=None):
    connection = oracledb.connect(
        user="ADMIN",
        password="TecMonterreyTeam3",
        dsn="team3vectordatabase_high",
        config_dir="/content/wallet"
    )

    with connection.cursor() as cursor:
        cursor.execute(sql, params or {})
        result = cursor.fetchall()

    connection.close()
    return result


## El Cerebro Lógico: Clase RetailIntelligenceBackend
Encapsulamos toda la lógica en una clase robusta diseñada para ser desplegada posteriormente como una API.

## Arquitectura del Backend


La clase `RetailIntelligenceBackend` integra tres capacidades principales:


1. **Recomendación Personalizada** (recommend_items_for_user): Utiliza el modelo basado en embeddings cargado en memoria para predecir el nivel de interés de un usuario por productos específicos.


2. **Búsqueda Semántica** (search_by_text): Convierte la consulta del usuario en un vector usando OCI GenAI y realiza una búsqueda de distancia (**VECTOR_DISTANCE**) en la base de datos para encontrar productos conceptualmente similares.


3. **Búsqueda Híbrida** (hybrid_search): Combina los resultados semánticos con las predicciones de gusto personal, aplicando una fórmula de ponderación dinámica para reordenar los resultados. Esto asegura que el usuario encuentre lo que busca, pero priorizando el estilo que le gusta.

In [ ]:
class RetailIntelligenceBackend:
    def __init__(self, model_path, db_params, genai_client, compartment_id):

        # Cargamos embeddings precomputados
        try:
            model_data = joblib.load(model_path)
            self.article_ids = model_data["article_ids"]
            self.article_vectors = model_data["article_vectors"]
            # índice rápido article_id - posición
            self.article_id_to_index = {
                int(aid): idx for idx, aid in enumerate(self.article_ids)
            }
            print(f"Modelo content-based cargado en memoria ({len(self.article_ids)} artículos).")
        except Exception as e:
            print(f"Error cargando modelo content-based: {e}")
            self.article_ids = np.array([])
            self.article_vectors = np.zeros((0, 0))
            self.article_id_to_index = {}

        self.db_params = db_params
        self.genai_client = genai_client
        self.compartment_id = compartment_id

    def get_db_connection(self):
        return oracledb.connect(**self.db_params)

    def _extract_field_from_rag(self, rag_text, field_prefix):
        if not rag_text:
            return "Desconocido"
        try:
            parts = rag_text.split(';')
            for part in parts:
                if field_prefix in part:
                    return part.replace(field_prefix, '').strip()
            return "General"
        except:
            return "General"

    # Funciones del modelo
    def get_top_articles_for_user(self, customer_id, top_n=20):
        sql = """
            SELECT "article_id", "purchase_count"
            FROM INTERACTIONS_FOR_CF
            WHERE "customer_id" = :user_id
            ORDER BY "purchase_count" DESC, "article_id" ASC
            FETCH FIRST :top_n ROWS ONLY
        """
        conn = self.get_db_connection()
        cursor = conn.cursor()
        try:
            cursor.execute(sql, user_id=customer_id, top_n=top_n)
            rows = cursor.fetchall()
            df = pd.DataFrame(rows, columns=["article_id", "purchase_count"])
            return df
        finally:
            cursor.close()
            conn.close()

    def build_user_profile_vector(self, top_df):

        # top_df: DataFrame con columnas ["article_id", "purchase_count"]
        # Usamos self.article_vectors (ya cargados del modelo) para construir el vector del usuario

        vecs = []
        weights = []
        for _, row in top_df.iterrows():
            aid = int(row["article_id"])
            idx = self.article_id_to_index.get(aid)
            if idx is None:
                continue
            vecs.append(self.article_vectors[idx])
            weights.append(row["purchase_count"])

        if not vecs:
            return None

        item_vecs = np.vstack(vecs)
        weights = np.array(weights, dtype=np.float32)

        user_vec = np.average(item_vecs, axis=0, weights=weights)

        # normalizar
        norm = np.linalg.norm(user_vec)
        if norm > 0:
            user_vec = user_vec / norm

        return user_vec

    def get_user_profile(self, customer_id, top_n=20):
        top_df = self.get_top_articles_for_user(customer_id, top_n=top_n)
        if top_df.empty:
            return None
        return self.build_user_profile_vector(top_df)


    # FUNCIÓN RECOMENDACIÓN PERSONALIZADA
    def recommend_items_for_user(self, customer_id, top_n=5):
        print(f"Generando recomendaciones para usuario: {customer_id[:10]}...")

        # Vector de gustos del usuario
        user_vec = self.get_user_profile(customer_id, top_n=20)
        if user_vec is None:
            print("Usuario sin suficientes datos para construir perfil.")
            return []

        if self.article_vectors.shape[0] == 0:
            print("No hay embeddings cargados en el modelo.")
            return []

        # Similitud coseno = producto punto
        sims = self.article_vectors @ user_vec

        # Top N artículos más similares
        top_idx = np.argsort(-sims)[:top_n]
        rec_ids = self.article_ids[top_idx]
        rec_scores = sims[top_idx]

        # Enriquecer con RAG y URL de imagen desde ARTICLES_MODIFIED
        conn = self.get_db_connection()
        cursor = conn.cursor()
        try:
            id_list = ",".join(str(int(a)) for a in rec_ids)
            sql_meta = f"""
                SELECT "article_id", "description_vector_rag", "img_url_team3"
                FROM ARTICLES_MODIFIED
                WHERE "article_id" IN ({id_list})
            """
            cursor.execute(sql_meta)
            meta_rows = cursor.fetchall()

            # indexamos score por id para fácil merge
            score_by_id = {int(a): float(s) for a, s in zip(rec_ids, rec_scores)}

            scored_items = []
            for art_id, rag_text, img_url in meta_rows:
                product_name = self._extract_field_from_rag(rag_text, "NOMBRE: ")
                product_group = self._extract_field_from_rag(rag_text, "GRUPO: ")
                product_color = self._extract_field_from_rag(rag_text, "COLOR: ")

                scored_items.append({
                    "article_id": art_id,
                    "name": product_name,
                    "group": product_group,
                    "color": product_color,
                    "image_url": img_url,
                    "score": round(score_by_id.get(int(art_id), 0.0), 4)
                })

            # Ordenamos por score
            scored_items.sort(key=lambda x: x['score'], reverse=True)
            return scored_items[:top_n]

        except Exception as e:
            print(f"Error en recomendación content-based: {e}")
            return []
        finally:
            cursor.close()
            conn.close()

    # FUNCIÓN BÚSQUEDA SEMÁNTICA (EMBEDDINGS)
    def search_by_text(self, query_text, top_n=5):
        print(f"Buscando semánticamente: '{query_text}'")

        try:
            embed_details = oci.generative_ai_inference.models.EmbedTextDetails(
                inputs=[query_text],
                serving_mode=oci.generative_ai_inference.models.OnDemandServingMode(
                    model_id="cohere.embed-english-v3.0"
                ),
                truncate="END",
                compartment_id=self.compartment_id
            )
            response = self.genai_client.embed_text(embed_details)
            query_vector = response.data.embeddings[0]
        except Exception as e:
            print(f"Error GenAI: {e}")
            return []

        conn = self.get_db_connection()
        cursor = conn.cursor()

        results = []
        try:
            sql_vector = """
                SELECT a."article_id", a."description_vector_rag", a."img_url_team3",
                       VECTOR_DISTANCE(e."vector", :qv) as dist
                FROM ARTICLE_EMBEDDINGS e
                JOIN ARTICLES_MODIFIED a ON e."article_id" = a."article_id"
                ORDER BY dist ASC
                FETCH FIRST :top_n ROWS ONLY
            """

            vector_array = array.array("f", query_vector)
            cursor.execute(sql_vector, qv=vector_array, top_n=top_n)

            for row in cursor:
                rag_text = row[1]

                product_name = self._extract_field_from_rag(rag_text, "NOMBRE: ")
                product_group = self._extract_field_from_rag(rag_text, "GRUPO: ")
                product_color = self._extract_field_from_rag(rag_text, "COLOR: ")

                results.append({
                    "article_id": row[0],
                    "name": product_name,
                    "group": product_group,
                    "color": product_color,
                    "image_url": row[2],
                    "similarity_score": round(1 - row[3], 4)
                })
            return results
        except Exception as e:
            print(f"Error en búsqueda vectorial: {e}")
            return []
        finally:
            cursor.close()
            conn.close()

    # FUNCIÓN BÚSQUEDA HÍBRIDA (texto + perfil de usuario con embeddings)
    def hybrid_search(self, customer_id, query_text, top_n=5, weight_text=0.7, weight_user=0.3):
        print(f"Búsqueda Híbrida para {customer_id[:8]}...: '{query_text}'")

        # Candidatos por búsqueda semántica
        semantic_results = self.search_by_text(query_text, top_n=50)
        if not semantic_results:
            return []

        # Vector de usuario (perfil)
        user_vec = self.get_user_profile(customer_id, top_n=20)
        if user_vec is None:
            # si no hay perfil, devolvemos solo semántico
            return semantic_results[:top_n]

        hybrid_results = []
        max_personal = 0.0
        temp_items = []

        # Calculamos "personal_score" como similitud coseno entre user_vec y embedding del artículo
        for item in semantic_results:
            aid = int(item['article_id'])
            idx = self.article_id_to_index.get(aid)
            if idx is None:
                personal_score = 0.0
            else:
                personal_score = float(self.article_vectors[idx] @ user_vec)

            if personal_score > max_personal:
                max_personal = personal_score

            temp_items.append({**item, "raw_personal": personal_score})

        if max_personal == 0:
            max_personal = 1.0

        # Combinamos semántico + perfil usuario
        for item in temp_items:
            normalized_personal = item['raw_personal'] / max_personal
            semantic_score = item.get('similarity_score', 0.0)

            final_score = (weight_text * semantic_score) + (weight_user * normalized_personal)

            hybrid_results.append({
                "article_id": item['article_id'],
                "name": item['name'],
                "group": item['group'],
                "color": item['color'],
                "image_url": item['image_url'],
                "semantic_score": round(semantic_score, 4),
                "personal_score": round(item['raw_personal'], 4),
                "final_score": round(final_score, 4)
            })

        hybrid_results.sort(key=lambda x: x['final_score'], reverse=True)
        return hybrid_results[:top_n]

    # FUNCIÓN RECOMENDACIONES RELACIONADAS AL PRODUCTO (Item-to-Item + User)
    # Para: "Productos que te podrían interesar" al dar click en un producto
    def recommend_related_items(self, customer_id, anchor_article_id, top_n=5, weight_item=0.8, weight_user=0.2):
        print(f"Buscando similares a producto {anchor_article_id} para usuario {customer_id[:8]}...")

        # Obtenemos vector del producto ANCLA
        anchor_idx = self.article_id_to_index.get(int(anchor_article_id))
        if anchor_idx is None:
            print("Producto ancla no encontrado en el modelo.")
            return []

        anchor_vec = self.article_vectors[anchor_idx] # Vector del producto clickeado

        # Obtenemos vector del USUARIO
        user_vec = self.get_user_profile(customer_id) # Puede ser None

        # Calculamos Similitud con el producto (Item-to-Item)
        sims_item = self.article_vectors @ anchor_vec

        # Similitud con el usuario (Personalización)
        if user_vec is not None:
            sims_user = self.article_vectors @ user_vec
        else:
            sims_user = np.zeros_like(sims_item) # Si no hay usuario, solo cuenta el item

        # Combinamos Scores
        # final = w1 * (Parecido al producto) + w2 * (Parecido a mis gustos)
        final_scores = (weight_item * sims_item) + (weight_user * sims_user)

        # Filtramos y ordenamos
        # Queremos excluir el mismo producto que estamos viendo
        final_scores[anchor_idx] = -1.0

        top_idx = np.argsort(-final_scores)[:top_n]
        rec_ids = self.article_ids[top_idx]
        rec_scores = final_scores[top_idx]

        return self._enrich_response(rec_ids, rec_scores)


    # --- HELPERS ---
    def _enrich_response(self, rec_ids, rec_scores):
        """Función auxiliar para ir a la BD por detalles dado una lista de IDs y Scores."""
        if len(rec_ids) == 0: return []

        conn = self.get_db_connection()
        cursor = conn.cursor()
        try:
            id_list = ",".join(str(int(a)) for a in rec_ids)
            sql_meta = f"""
                SELECT "article_id", "description_vector_rag", "img_url_team3"
                FROM ARTICLES_MODIFIED
                WHERE "article_id" IN ({id_list})
            """
            cursor.execute(sql_meta)
            meta_rows = cursor.fetchall()

            score_map = {int(a): float(s) for a, s in zip(rec_ids, rec_scores)}

            results = []
            for art_id, rag_text, img_url in meta_rows:
                results.append({
                    "article_id": art_id,
                    "name": self._extract_field_from_rag(rag_text, "NOMBRE: "),
                    "group": self._extract_field_from_rag(rag_text, "GRUPO: "),
                    "color": self._extract_field_from_rag(rag_text, "COLOR: "),
                    "image_url": img_url,
                    "score": round(score_map.get(int(art_id), 0.0), 4)
                })

            # Reordenamos
            results.sort(key=lambda x: x['score'], reverse=True)
            return results

        except Exception as e:
            print(f"Error enriqueciendo datos: {e}")
            return []
        finally:
            cursor.close()
            conn.close()

## Instanciación y Pruebas Unitarias
Validamos que el backend funcione correctamente antes de su despliegue.

### Instanciación del Backend


Creamos una instancia de la clase pasando las rutas de los modelos y las credenciales de base de datos. Confirmamos que el cliente de IA Generativa esté configurado correctamente para inferencia.

In [ ]:
# --- INSTANCIAMOS EL BACKEND ---
backend = RetailIntelligenceBackend(
    model_path=LOCAL_MODEL_PATH,
    db_params=db_config,
    genai_client=genai_inference_client,
    compartment_id=GENAI_COMPARTMENT_ID
)

Modelo content-based cargado en memoria (105542 artículos).


## Prueba de Capacidades


Ejecutamos tres pruebas clave:


1. **Recomendación Pura**: Verificamos que el sistema sugiera productos al usuario basándose solo en su historial (top 20 compras).


2. **Búsqueda Semántica**: Probamos con una frase compleja ("An elegant red dress for a party at night") para validar que el motor entiende el contexto.


3. **Búsqueda Híbrida**: Comprobamos que el sistema reordena los resultados semánticos basándose en el perfil del usuario (el puntaje "Gusto" altera el ranking final).

**Prueba A**: Recomendación Personalizada (Embeddings).

Vamos a pedirle recomendaciones para un usuario dentro de la base de datos.

In [ ]:
# Tomamos un customer_id de prueba desde la BD
test_user_id = query('SELECT "customer_id" FROM CUSTOMERS_MODIFIED FETCH FIRST 1 ROWS ONLY')[0][0]

print(f"Usuario de prueba: {test_user_id}")

# Llamada al backend
recommendations = backend.recommend_items_for_user(test_user_id, top_n=5)

print("\n--- RECOMENDACIONES PERSONALIZADAS (HISTORIAL DEL USUARIO) ---")
for item in recommendations:
    print(f" Producto: {item['name']}")
    print(f"   Categoría: {item['group']} |  Color: {item['color']}")
    print(f"   Score similitud: {item['score']}")
    print(f"   Img: {item['image_url']}")
    print("-" * 30)


Usuario de prueba: 91dc9144398099a88f311f5f653d97ca199236d12be865d100c92d0da3c541f9
Generando recomendaciones para usuario: 91dc914439...

--- RECOMENDACIONES PERSONALIZADAS (HISTORIAL DEL USUARIO) ---
 Producto: Doutzen
   Categoría: Garment Upper body |  Color: Dark Beige
   Score similitud: 1.0
   Img: https://objectstorage.us-chicago-1.oraclecloud.com/n/axym8lqm5eyc/b/bucketia/o/images/066/0668537001.jpg
------------------------------
 Producto: Doutzen
   Categoría: Garment Upper body |  Color: Black
   Score similitud: 0.9715
   Img: https://objectstorage.us-chicago-1.oraclecloud.com/n/axym8lqm5eyc/b/bucketia/o/images/066/0668537002.jpg
------------------------------
 Producto: L Stitch Pile Coat
   Categoría: Garment Upper body |  Color: Yellowish Brown
   Score similitud: 0.8336
   Img: https://objectstorage.us-chicago-1.oraclecloud.com/n/axheitjen1w2/b/reto/o/images/077/0772829002.jpg
------------------------------
 Producto: L Stitch Pile Coat
   Categoría: Garment Upper body

**Prueba B**: Búsqueda Semántica (Embeddings): Aquí buscaremos algo que no sea una palabra clave exacta, sino un concepto.

In [ ]:
# Prueba de búsqueda natural
search_query = "An elegant red dress for a party at night"

# Llamada al backend usando embeddings
results = backend.search_by_text(search_query, top_n=5)

print(f"\n--- BÚSQUEDA SEMÁNTICA (EMBEDDINGS): '{search_query}' ---")
for item in results:
    print(f" Producto: {item['name']}")
    print(f"   Categoría: {item['group']} |  Color: {item['color']}")
    print(f"   Similitud: {item['similarity_score']}")
    print(f"   Img: {item['image_url']}")
    print("-" * 30)


Buscando semánticamente: 'An elegant red dress for a party at night'

--- BÚSQUEDA SEMÁNTICA (EMBEDDINGS): 'An elegant red dress for a party at night' ---
 Producto: PARTY Harlow dress
   Categoría: Garment Full body |  Color: Red
   Similitud: 0.4976
   Img: https://objectstorage.us-chicago-1.oraclecloud.com/n/axheitjen1w2/b/reto/o/images/079/0796673002.jpg
------------------------------
 Producto: EVE party dress SET
   Categoría: Garment Full body |  Color: Dark Red
   Similitud: 0.4933
   Img: https://objectstorage.us-chicago-1.oraclecloud.com/n/axym8lqm5eyc/b/bucketia/o/images/064/0648650001.jpg
------------------------------
 Producto: Sam party dress
   Categoría: Garment Full body |  Color: Red
   Similitud: 0.4877
   Img: https://objectstorage.us-chicago-1.oraclecloud.com/n/axym8lqm5eyc/b/bucketia/o/images/067/0671302001.jpg
------------------------------
 Producto: CNY partydress
   Categoría: Garment Full body |  Color: Dark Red
   Similitud: 0.4848
   Img: https://objectsto

**Prueba C**: Ambos modelos unidos: Se utiliza la busqueda semantica para traer los productos mas similares semanticamente y utilizando Embeddings seleccionamos los productos que mas le podrian interesar al usuario con base en sus gustos

In [ ]:
# Usuario y texto de prueba
test_user_id = query('SELECT "customer_id" FROM CUSTOMERS_MODIFIED FETCH FIRST 1 ROWS ONLY')[0][0]
search_query = "An elegant red dress for a party at night"

print(f"Usuario de prueba: {test_user_id}")
print(f"Texto de búsqueda: '{search_query}'")

# Llamada al backend: híbrido (texto + historial)
results = backend.hybrid_search(
    customer_id=test_user_id,
    query_text=search_query,
    top_n=3,
    weight_text=0.7,   # peso del significado del texto
    weight_user=0.3    # peso del perfil del usuario
)

print(f"\n--- BÚSQUEDA HÍBRIDA (TEXTO + HISTORIAL) ---")
for item in results:
    print(f" Producto: {item['name']}")
    print(f"   Categoría: {item['group']} |  Color: {item['color']}")
    print(f"   Score semántico (texto): {item['semantic_score']}")
    print(f"   Score personal (usuario): {item['personal_score']}")
    print(f"   Score final combinado:    {item['final_score']}")
    print(f"   Img: {item['image_url']}")
    print("-" * 40)


Usuario de prueba: 0e5c9909c03c8a37991870114712c46b3ae0e7902220d8c4bb2a5390f35fed43
Texto de búsqueda: 'An elegant red dress for a party at night'
Búsqueda Híbrida para 0e5c9909...: 'An elegant red dress for a party at night'
Buscando semánticamente: 'An elegant red dress for a party at night'

--- BÚSQUEDA HÍBRIDA (TEXTO + HISTORIAL) ---
 Producto: Moonlight dress
   Categoría: Garment Full body |  Color: Dark Red
   Score semántico (texto): 0.4766
   Score personal (usuario): 0.7583
   Score final combinado:    0.6336
   Img: https://objectstorage.us-chicago-1.oraclecloud.com/n/axym8lqm5eyc/b/bucketia/o/images/054/0542443004.jpg
----------------------------------------
 Producto: CHRISTMAS nightgown
   Categoría: Nightwear |  Color: Dark Red
   Score semántico (texto): 0.4765
   Score personal (usuario): 0.7216
   Score final combinado:    0.619
   Img: https://objectstorage.us-chicago-1.oraclecloud.com/n/axym8lqm5eyc/b/bucketia/o/images/059/0595569001.jpg
---------------------------

**Prueba D**: Se utiliza la similitud entre productos para traer los productos más similares utilizando Embeddings y con el gusto personal seleccionamos los productos que más le podrían interesar al usuario para re-ordenar estos productos.

In [ ]:
# Prueba de la funcionalidad Item-to-Item
# Usamos el usuario de prueba
test_user_id = "0e5c9909c03c8a37991870114712c46b3ae0e7902220d8c4bb2a5390f35fed43"

# Simulamos que el usuario dio click en el "PARTY Harlow dress"
clicked_product_id = 796673002

# Llamada a la nueva función
related_items = backend.recommend_related_items(
    customer_id=test_user_id,
    anchor_article_id=clicked_product_id,
    top_n=5,
    weight_item=0.8, # Priorizamos que se parezca al producto
    weight_user=0.2  # Pero con un toque del gusto del usuario
)

print(f"\n--- PRODUCTOS RELACIONADOS (Porque viste el {clicked_product_id}) ---")
for item in related_items:
    print(f" Producto: {item['name']}")
    print(f"   Categoría: {item['group']} | Color: {item['color']}")
    print(f"   Score Híbrido: {item['score']}")
    print(f"   Img: {item['image_url']}")
    print("-" * 30)

Buscando similares a producto 796673002 para usuario 0e5c9909...

--- PRODUCTOS RELACIONADOS (Porque viste el 796673002) ---
 Producto: PARTY Harlow dress
   Categoría: Garment Full body | Color: Black
   Score Híbrido: 0.9193
   Img: https://objectstorage.us-chicago-1.oraclecloud.com/n/axheitjen1w2/b/reto/o/images/079/0796673001.jpg
------------------------------
 Producto: Harlow dress
   Categoría: Garment Full body | Color: White
   Score Híbrido: 0.7785
   Img: https://objectstorage.us-chicago-1.oraclecloud.com/n/axym8lqm5eyc/b/bucketia/o/images/054/0549527001.jpg
------------------------------
 Producto: PARTY Love dress
   Categoría: Garment Full body | Color: Dark Pink
   Score Híbrido: 0.772
   Img: https://objectstorage.us-chicago-1.oraclecloud.com/n/axheitjen1w2/b/reto/o/images/079/0796646001.jpg
------------------------------
 Producto: Make a scene dress
   Categoría: Garment Full body | Color: Dark Red
   Score Híbrido: 0.7639
   Img: https://objectstorage.us-chicago-1.or